# Clustering Similar Neighborhoods based

#### Import Libraries

In [68]:
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import folium 
import requests
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Reading data with Lat/Long Information

In [3]:
df = pd.read_pickle("NeigborhoodCoordinates.pkl")

In [4]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Reading Neighborhood data from FourSquare API for a single PoI

In [5]:
CLIENT_ID = 'JRNMSKYBV443Z1P0CTJU3PYONLTZE3M4XDUXTMCSMYPYYQGZ' # your Foursquare ID
CLIENT_SECRET = 'FXV0DZFNZLTYS2MGZD4PM3I4L5MUOE0KI2W4VL3OBWGWWGJA' # your Foursquare Secret
VERSION = '20200726' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JRNMSKYBV443Z1P0CTJU3PYONLTZE3M4XDUXTMCSMYPYYQGZ
CLIENT_SECRET:FXV0DZFNZLTYS2MGZD4PM3I4L5MUOE0KI2W4VL3OBWGWWGJA


In [6]:
print('Latitude and longitude values of {}, {} with postal code {} are {}, {}.'.format(df.Neighborhood[0], df.Borough[0],df['Postal Code'][0],df.Latitude[0],
                                                                                      df.Longitude[0]))

Latitude and longitude values of Parkwoods, North York with postal code M3A are 43.7532586, -79.3296565.


In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df.Latitude[0], 
    df.Longitude[0], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JRNMSKYBV443Z1P0CTJU3PYONLTZE3M4XDUXTMCSMYPYYQGZ&client_secret=FXV0DZFNZLTYS2MGZD4PM3I4L5MUOE0KI2W4VL3OBWGWWGJA&v=20200726&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1db148eb4b917f81d4508f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

#### Converting API data into dataframe

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


#### Getting API data for all PoI and converting to DataFrame 

In [38]:
def getNearbyVenues(postalcode, borough, neighborhood, latitudes, longitudes, radius=500):
    count=0
    venues_list=[]
    for postalcode, lat, lng in zip(postalcode, latitudes, longitudes):
        count=count+1
        print(postalcode,count)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode, borough, neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 'Borough', 'NHood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
toronto_venues = getNearbyVenues(postalcode=df['Postal Code'],borough=df.Borough, neighborhood=df.Neighborhood,
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

M3A 1
M4A 2
M5A 3
M6A 4
M7A 5
M9A 6
M1B 7
M3B 8
M4B 9
M5B 10
M6B 11
M9B 12
M1C 13
M3C 14
M4C 15
M5C 16
M6C 17
M9C 18
M1E 19
M4E 20
M5E 21
M6E 22
M1G 23
M4G 24
M5G 25
M6G 26
M1H 27
M2H 28
M3H 29
M4H 30
M5H 31
M6H 32
M1J 33
M2J 34
M3J 35
M4J 36
M5J 37
M6J 38
M1K 39
M2K 40
M3K 41
M4K 42
M5K 43
M6K 44
M1L 45
M2L 46
M3L 47
M4L 48
M5L 49
M6L 50
M9L 51
M1M 52
M2M 53
M3M 54
M4M 55
M5M 56
M6M 57
M9M 58
M1N 59
M2N 60
M3N 61
M4N 62
M5N 63
M6N 64
M9N 65
M1P 66
M2P 67
M4P 68
M5P 69
M6P 70
M9P 71
M1R 72
M2R 73
M4R 74
M5R 75
M6R 76
M7R 77
M9R 78
M1S 79
M4S 80
M5S 81
M6S 82
M1T 83
M4T 84
M5T 85
M1V 86
M4V 87
M5V 88
M8V 89
M9V 90
M1W 91
M4W 92
M5W 93
M8W 94
M9W 95
M1X 96
M4X 97
M5X 98
M8X 99
M4Y 100
M7Y 101
M8Y 102
M8Z 103


In [44]:
toronto_venues.drop(['Borough','NHood'],axis=1,inplace=True)

In [45]:
toronto_venues.shape

(2153, 7)

In [46]:
toronto_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,2,2,2,2,2,2
M1C,1,1,1,1,1,1
M1E,8,8,8,8,8,8
M1G,4,4,4,4,4,4
M1H,8,8,8,8,8,8
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,8,8,8,8,8,8
M9R,4,4,4,4,4,4


In [47]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


#### Creating one hot encoding & finding means of nearby establishments for clustering

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 


# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(20)

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head(20)

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,M1J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,M1K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,M1L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,M1M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,M1N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
toronto_grouped.shape

(100, 270)

#### Top 3 venues by PoI

In [57]:
num_top_venues = 3

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Print Shop   0.5
2     Martial Arts Dojo   0.0


----M1C----
                venue  freq
0                 Bar   1.0
1   Accessories Store   0.0
2  Miscellaneous Shop   0.0


----M1E----
                venue  freq
0      Medical Center  0.12
1  Mexican Restaurant  0.12
2        Intersection  0.12


----M1G----
               venue  freq
0        Coffee Shop  0.50
1       Soccer Field  0.25
2  Korean Restaurant  0.25


----M1H----
                 venue  freq
0  Fried Chicken Joint  0.12
1          Gas Station  0.12
2               Bakery  0.12


----M1J----
         venue  freq
0   Playground   0.5
1  Pizza Place   0.5
2     Pharmacy   0.0


----M1K----
              venue  freq
0    Discount Store   0.4
1  Department Store   0.2
2        Hobby Shop   0.2


----M1L----
          venue  freq
0      Bus Line   0.2
1        Bakery   0.2
2  Soccer Field   0.1


----M1M----
                       venue  fr

#### Finding top 10 venue categories for each PoI

In [111]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,Mexican Restaurant,Electronics Store,Restaurant,Breakfast Spot,Rental Car Location,Medical Center,Bank,Intersection,Yoga Studio,Discount Store
3,M1G,Coffee Shop,Soccer Field,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M1H,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Discount Store,Dim Sum Restaurant


#### Clustering  PoIs basis the venue categories in the neighborhood

In [113]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

df_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Furniture / Home Store,Women's Store,Vietnamese Restaurant,Convenience Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park


In [115]:
df_final=df_merged[df_merged['Cluster Labels'].isna()==False]

#### Visualizing clusters on Map

In [116]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto, Ontario')
latitude_city=location.latitude
longitude_city=location.longitude
print('The co-ordinates of Toronto, ON are {} latitude and {} longitude.'.format(location.latitude,location.longitude))

The co-ordinates of Toronto, ON are 43.6534817 latitude and -79.3839347 longitude.


In [124]:
# create map
map_clusters = folium.Map(location=[latitude_city, longitude_city], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Postal Code'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Check Individual Clusters for similarities

In [118]:
df_final.loc[df_final['Cluster Labels'] == 0, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,0.0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
94,Etobicoke,0.0,Rental Car Location,Bar,Drugstore,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


####  Cluster 0 : Area differentiated by Bars and Rental Car Locations

In [119]:
df_final.loc[df_final['Cluster Labels'] == 1, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
16,York,1.0,Trail,Park,Field,Hockey Arena,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,York,1.0,Park,Women's Store,Pool,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
35,East York,1.0,Park,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
40,North York,1.0,Airport,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
52,North York,1.0,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
64,York,1.0,Park,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
66,North York,1.0,Park,Construction & Landscaping,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
68,Central Toronto,1.0,Jewelry Store,Park,Sushi Restaurant,Trail,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
85,Scarborough,1.0,Park,Playground,Coffee Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


#### Cluster 1: Area differentiated by Parks, Playgrounds

In [120]:
df_final.loc[df_final['Cluster Labels'] == 2, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,2.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
3,North York,2.0,Clothing Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Furniture / Home Store,Women's Store,Vietnamese Restaurant,Convenience Store
4,Downtown Toronto,2.0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
6,Scarborough,2.0,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,2.0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,American Restaurant,Steakhouse,Asian Restaurant,Seafood Restaurant
98,Etobicoke,2.0,River,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Department Store
99,Downtown Toronto,2.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Bubble Tea Shop,Dance Studio,Mediterranean Restaurant,Men's Store
100,East Toronto,2.0,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant


#### Cluster 2:  Biggest Cluster: Differentiated by the presence of Coffee Shops and dominated by speciality restaurants/eateries

In [121]:
df_final.loc[df_final['Cluster Labels'] == 3, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Fabric Shop,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
101,Etobicoke,3.0,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market


#### Cluster 3: Area Differentiated by Baseball Fields

In [122]:
df_final.loc[df_final['Cluster Labels'] == 4, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,4.0,Jewelry Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market


#### Cluster 4: Single Area with proximity to Jewelry Stores and Yoga Studios